In [1]:
import findspark

findspark.init()

from pyspark import SparkContext, SparkConf
import json

def json_parser(line):
    parser = json.loads(line)
    return parser

def serialize(line):
    encoder = json.dumps(line, sort_keys = True)
    return encoder

def reducer(x,y):
    sum_scores = x[0] + y[0]
    count = x[1] + y[1]
    return sum_scores,count

def cal_avg(kv):
    k, (sum_score, count) = kv
    return (k, sum_score/count)

def rel_score(kv):
    k, (avg, comm_score) = kv
    return k,comm_score/avg

def max_score(x,y):
    if y>x:
        return y
    else:
        return x

def get_key(kv):
    return kv[1]
    

conf = SparkConf().setAppName('reddit_av')

sc = SparkContext(conf=conf)

rdd0 = sc.textFile('../../Downloads/reddit-1')

rdd1= rdd0.map(json_parser).cache()
commentbysub = rdd1.map(lambda c: (c['subreddit'], c))

rdd2= rdd1.map(lambda line:(line['subreddit'],(line['score'],1)))
rdd3= rdd2.reduceByKey(reducer)
rdd4 = rdd3.map(cal_avg)
filtered_rdd4 = rdd4.filter(lambda line:line[1] > 0)

In [2]:
pairRDD = filtered_rdd4.join(commentbysub)

In [3]:
rdd3.take(10)

[('scala', (1873, 971)),
 ('Genealogy', (1396, 746)),
 ('xkcd', (38329, 7269)),
 ('Cameras', (11, 9)),
 ('optometry', (222, 151))]

In [4]:
finalRDD = pairRDD.map(lambda line: (line[1][1]['author'],(line[1][0],line[1][1]['score'])))

In [5]:
finalRDD1 = finalRDD.map(rel_score)

In [14]:
print(finalRDD1.filter(lambda line: line[0]=="jankotek").take(10))

[('jankotek', -4.147357180993059)]


In [6]:
finalRDD2 = finalRDD1.reduceByKey(max_score).sortByKey()

In [7]:
f3 = finalRDD2.sortBy(get_key, ascending= True)

In [8]:
f3.take(10)

[('jankotek', -4.147357180993059),
 ('andyzweb', -3.224007931331368),
 ('igkunow', -3.224007931331368),
 ('pragmatist', -3.0343604059589344),
 ('Golden_Bender', -2.0861227790967676),
 ('patchbag', -1.896475253724334),
 ('stringerbell', -1.896475253724334),
 ('voiceofdissent', -1.896475253724334),
 ('matrixifyme', -1.7068277283519007),
 ('wza', -1.6031518624641834)]